In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
import itertools

warnings.filterwarnings('ignore')

possible_n_vals = [10,12,14,16,18,20]

def run_grid_search_nn(n, param_grid=None, max_combinations=None, verbose=True):
    """Load data for n, split, scale, then run grid search over parameter grid.
    param_grid should be a dict of lists, e.g.:
      {'hidden_layer_sizes': [(16,),(64,),(128,64)], 'activation': ['relu','tanh'], 'alpha':[1e-4,1e-3], 'solver':['adam','sgd'], 'max_iter':[300,500]}
    max_combinations: optional int to limit tested combos (first N) to avoid long runs.
    Returns list of result dicts sorted by validation accuracy (desc)."""

    X = np.load('Datasets/kryptonite-%s-X.npy' % (n))
    y = np.load('Datasets/kryptonite-%s-y.npy' % (n))

    # Split: 60% train, 20% val, 20% test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)

    # Default parameter grid if none supplied
    if param_grid is None:
        param_grid = {
            'hidden_layer_sizes': [(64,), (128,64,32)],
            'activation': ['tanh'],
            'alpha': [1e-4],
            'solver': ['adam'],
            'max_iter': [500]
        }

    # Prepare combinations
    keys = list(param_grid.keys())
    values = [param_grid[k] for k in keys]
    combos = list(itertools.product(*values))
    if max_combinations is not None:
        combos = combos[:max_combinations]

    results = []
    for idx, combo in enumerate(combos, start=1):
        params = dict(zip(keys, combo))
        # ensure types are correct (hidden_layer_sizes must be tuple already in grid)
        clf_params = {k: v for k, v in params.items() if k != 'max_iter'}
        clf_params['max_iter'] = params.get('max_iter', 300)
        clf_params['random_state'] = 42

        name = f"model_{idx}"
        if verbose:
            print(f"[{idx}/{len(combos)}] Testing {name} with {clf_params} ...")

        clf = MLPClassifier(**clf_params)
        try:
            clf.fit(X_train, y_train)
        except Exception as e:
            # Skip combos that fail to train (e.g., convergence issues) and record the error
            if verbose:
                print(f"  Skipped {name} due to error: {e}")
            results.append({'name': name, 'params': clf_params, 'error': str(e)})
            continue

        # Evaluate
        y_train_pred = clf.predict(X_train)
        train_acc = accuracy_score(y_train, y_train_pred)
        y_val_pred = clf.predict(X_val)
        val_acc = accuracy_score(y_val, y_val_pred)
        y_test_pred = clf.predict(X_test)
        test_acc = accuracy_score(y_test, y_test_pred)

        if verbose:
            print(f"  {name}: Train={train_acc:.4f}, Val={val_acc:.4f}, Test={test_acc:.4f}")

        results.append({'name': name, 'params': clf_params, 'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc, 'model': clf})

    # Sort results by val_acc (missing val_acc go to bottom)
    sorted_results = sorted(results, key=lambda r: r.get('val_acc', -1), reverse=True)
    if verbose and sorted_results:
        best = sorted_results[0]
        if 'val_acc' in best:
            print('\nBest by validation: {} params={} val={:.4f} test={:.4f}'.format(best['name'], best['params'], best['val_acc'], best['test_acc']))
        else:
            print('\nNo successful trained models found.')

    return sorted_results

# Note: adjust param_grid and max_combinations to control search breadth and runtime.


In [ ]:
for n in possible_n_vals:
    # Example: limit to first 20 combinations to keep runtime reasonable
    results = run_grid_search_nn(n, max_combinations=20, verbose=True)
    # inspect top result if needed:
    # print(results[0])